In [ ]:
!git clone https://github.com/OpenBMB/cpm.cu.git --recursive
!cd cpm.cu
!python3 setup.py install

Cloning into 'cpm.cu'...
remote: Enumerating objects: 5304, done.
remote: Counting objects: 100% (391/391), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 5304 (delta 334), reused 322 (delta 316), pack-reused 4913 (from 2)
Receiving objects: 100% (5304/5304), 11.78 MiB | 15.27 MiB/s, done.
Resolving deltas: 100% (3869/3869), done.
Submodule 'src/cutlass' (https://github.com/NVIDIA/cutlass.git) registered for path 'src/cutlass'
Cloning into '/content/cpm.cu/src/cutlass'...
remote: Enumerating objects: 35458, done.        
remote: Counting objects: 100% (251/251), done.        
remote: Compressing objects: 100% (140/140), done.        
remote: Total 35458 (delta 129), reused 112 (delta 111), pack-reused 35207 (from 3)        
Receiving objects: 100% (35458/35458), 53.50 MiB | 22.33 MiB/s, done.
Resolving deltas: 100% (26650/26650), done.
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (12/12), done.
remote: Total 16 (delta 12), reused 12 (delta

In [ ]:
%cd cpm.cu
!python3 setup.py install

/content/cpm.cu
DEPENDENCY ERRORS AND INSTALLATION SUGGESTIONS:
   • Ninja is not installed. Required version: >=1.10.0
      1. Please install Ninja with version >=1.10.0 using: pip install 'ninja>=1.10.0'

   • CUDA is not available and CPMCU_CUDA_ARCH is not set
      1. Check GPU/driver installation first. Verify: python -c "import torch; print(torch.cuda.is_available())"
            IMPORTANT: Do NOT use CPMCU_CUDA_ARCH to bypass GPU detection issues.
            CPMCU_CUDA_ARCH is only for cross-platform builds without local GPU access.
            Details: python setup.py --help-config
      2. For new GPUs (RTX 5090/5070Ti): pip install --pre torch --index-url https://download.pytorch.org/whl/nightly/cu128
            Latest info: https://pytorch.org/get-started/locally/
      3. For Jetson devices: pip install 'torch>=2.0.0' --index-url https://pypi.jetson-ai-lab.dev/jp6/cu126
            Find compatible versions at: https://pypi.jetson-ai-lab.dev
            Alternative: Use 

https://huggingface.co/openbmb/MiniCPM4.1-8B

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch.manual_seed(0)

path = 'openbmb/MiniCPM4.1-8B'
# device = "cpu"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForCausalLM.from_pretrained(path, torch_dtype=torch.bfloat16, device_map="auto", trust_remote_code=True)

# User can directly use the chat interface
# responds, history = model.chat(tokenizer, "Write an article about Artificial Intelligence.", temperature=0.7, top_p=0.7)
# print(responds)

# User can also use the generate interface
messages = [
    {"role": "user", "content": "Write an article about Artificial Intelligence."},
]
prompt_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([prompt_text], return_tensors="pt").to(model.device)

model_outputs = model.generate(
    **model_inputs,
    max_new_tokens=32,
    top_p=0.95,
    temperature=0.6
)
output_token_ids = [
    model_outputs[i][len(model_inputs[i]):] for i in range(len(model_inputs['input_ids']))
]

responses = tokenizer.batch_decode(output_token_ids, skip_special_tokens=True)[0]
print(responses)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<think>
Okay, the user wants me to write an article about Artificial Intelligence. Let me start by understanding their needs. They might be a student


In [ ]:
2400/60


40.0

In [ ]:
!pip install llama-cpp-python

from llama_cpp import Llama

llm = Llama.from_pretrained(
        repo_id="openbmb/MiniCPM4.1-8B-GGUF",
        filename="MiniCPM4.1-8B-Q4_K_M.gguf",
)

./MiniCPM4.1-8B-Q4_K_M.gguf:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 33 key-value pairs and 293 tensors from /root/.cache/huggingface/hub/models--openbmb--MiniCPM4.1-8B-GGUF/snapshots/ebb834d0ad9acfa98bd16bc963ec51be5f7e08c1/./MiniCPM4.1-8B-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = minicpm
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Job_69208.Global_Step_25.Rl_Ifeval_Ta...
llama_model_loader: - kv   3:                         general.size_label str              = 8.2B
llama_model_loader: - kv   4:                        minicpm.block_count u32              = 32
llama_model_loader: - kv   5:                     minicpm.context_length u32              = 65536
llama_model_loader: - kv   6

In [ ]:
llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": "What is the capital of France?"
		}
	]
)

llama_perf_context_print:        load time =   14614.35 ms
llama_perf_context_print: prompt eval time =   14614.07 ms /    16 tokens (  913.38 ms per token,     1.09 tokens per second)
llama_perf_context_print:        eval time =  144575.29 ms /   159 runs   (  909.28 ms per token,     1.10 tokens per second)
llama_perf_context_print:       total time =  159383.67 ms /   175 tokens
llama_perf_context_print:    graphs reused =        153


{'id': 'chatcmpl-a3d5d7fe-1517-4359-bd05-b2c3f352b962',
 'object': 'chat.completion',
 'created': 1757462309,
 'model': '/root/.cache/huggingface/hub/models--openbmb--MiniCPM4.1-8B-GGUF/snapshots/ebb834d0ad9acfa98bd16bc963ec51be5f7e08c1/./MiniCPM4.1-8B-Q4_K_M.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "<think>\nOkay, the user is asking about the capital of France. That's a straightforward geography question. \n\nFirst, I need to confirm the answer is Paris. No doubts there. \n\nBut the user might be testing basic knowledge or just needs quick info. Since they didn't add context, I'll keep it simple and direct. \n\nNo need for extra details unless they ask follow-ups. Just deliver the core answer cleanly. \n\nAlso, the tone should stay friendly and helpful—no robotic feel. They're probably just curious, so keep it approachable. \n\nYep, that covers it. Time to respond!\n</think>\nThe capital of France is **Paris**."},
   'logprobs': None,
   'fin